In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('retina')

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
from glob import glob
#plt.style.use('~/Desktop/saunders.mplstyle')

Start with just max daily temperature

In [3]:
days_to_drop = np.array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [4]:
stations = pd.read_fwf('data/ghcnd-stations.txt', colspecs=[[0, 11], [13, 20], [22, 30], [32, 37], [39, 40], [42, 71]],\
                       names=['station', 'latitude', 'longitude', 'elevation', 'state', 'name'])

In [5]:
filelist = glob('data/ghcnd_hcn/*.dly')

In [6]:
collect_tmax = []

for i, file in enumerate(filelist):
    print(i, '/', len(filelist), '\r', end='')
    
    data = pd.read_fwf(file,\
                       colspecs=[[0,11], [11, 15], [15, 17], [17, 21]] + \
                                 [[i, j] for i, j in zip(np.arange(21, 267, 8).tolist(), np.arange(26, 267, 8).tolist())], \
                       names=['station', 'year', 'month', 'type'] + ['day%s'%str(i) for i in range(1, 32)])

    data_tmax = data[data['type']=='TMAX']


    station = data.loc[0, 'station']
    years = np.unique(data_tmax['year'])
    
    station_info = stations[stations['station'] == station]

    for year in years:
        station_year = data_tmax[data_tmax['year']==year]
        if station_year.shape[0] < 12:
            continue
        else:
            collect_tmax.append([station, year] + station_year[station_year.columns[4:]].to_numpy().flatten()[~days_to_drop].tolist() + \
                                [station_info['latitude'].values[0], station_info['longitude'].values[0], station_info['elevation'].values[0]])
            
data_tmax_byyear = pd.DataFrame(data=collect_tmax, \
                                columns=['station', 'year'] + ['day%s'%str(i) for i in range(1, 366)] + ['latitude', 'longitude', 'elevation'])        

In [7]:
data_tmax_byyear.to_csv('data/ghcnd_hcn_tmax.csv.gz', compression='gzip', index=False)